# Utility

## Imports and Definitions

In [ ]:
import os
import torch
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt

from SCMP import SCMP
import DataGeneration as data
from CommonFunctions import *
from SocialMeasures import Utility

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)

PATH = "./Results/utility"
if not os.path.exists(PATH):
    os.makedirs(PATH)

## Data Loading

In [ ]:
# credit dataset
X, Y = data.load_credit_default_data()
X, Y = X[:3000], Y[:3000]
X, Y, Xval, Yval, Xtest, Ytest = data.split_validation_test(X, Y)

## Training

In [ ]:
x_dim = len(X[0])
scale = 1 # 1 / x_dim
epochs = 10
batch_size = 64

lambda_range = np.arange(0.05, 1.201, 0.05)
print(lambda_range)

In [ ]:
results = {
    "lambdas": [],
    "accuracies": [],
    "utilities": []
}

print(f"---------- Training with lambda={0} ----------")
model_name = f"utility_{0:.3f}"
utility = Utility(0, quad_cost_torch, cost_const_kwargs={"scale": scale})  # Create utility from outside since reg=0.
benchmark_model = SCMP(x_dim, batch_size, cost_fn="quad", cost_const_kwargs={"scale": scale}, 
                       social_measure_dict={"utility": utility}, strategic=True)
benchmark_model.fit(X, Y, Xval, Yval, opt_class=torch.optim.Adam, opt_kwargs={"lr": 5e-2}, epochs=epochs,
                    verbose="batches", path=PATH, model_name=model_name)

benchmark_model = SCMP(x_dim, batch_size, cost_fn="quad", cost_const_kwargs={"scale": scale}, 
                       social_measure_dict={"utility": utility}, strategic=True)
benchmark_model.load_model(PATH, model_name)

results["lambdas"].append(0)
accuracy = benchmark_model.evaluate(Xtest, Ytest, strategic_data=True)
results["accuracies"].append(accuracy)
Xtest_opt, Ytest_pred = benchmark_model.forward(Xtest, requires_grad=False)
utility = benchmark_model.social_measure_dict["utility"].calc_utility(Xtest, Xtest_opt, Ytest_pred, requires_grad=False).item()
results["utilities"].append(utility)
pd.DataFrame(results).to_csv(f"{PATH}/results.csv")

for lamb in lambda_range:
    print(f"---------- Training with lambda={lamb} ----------")
    model_name = f"utility_{lamb:.3f}"
    model = SCMP(x_dim, batch_size, cost_fn="quad", cost_const_kwargs={"scale": scale}, utility_reg=lamb, strategic=True)
    model.fit(X, Y, Xval, Yval, opt_class=torch.optim.Adam, opt_kwargs={"lr": 5e-2}, epochs=epochs,
              verbose="batches", path=PATH, model_name=model_name)
    
    model = SCMP(x_dim, batch_size, cost_fn="quad", cost_const_kwargs={"scale": scale}, utility_reg=lamb, strategic=True)
    model.load_model(PATH, model_name)
    
    # Calculate and save results.
    results["lambdas"].append(lamb)
    accuracy = model.evaluate(Xtest, Ytest, strategic_data=True)
    results["accuracies"].append(accuracy)
    Xtest_opt, Ytest_pred = model.forward(Xtest, requires_grad=False)
    utility = model.social_measure_dict["utility"].calc_utility(Xtest, Xtest_opt, Ytest_pred, requires_grad=False).item()
    results["utilities"].append(utility)
    pd.DataFrame(results).to_csv(f"{PATH}/results.csv")

## Show Results

In [ ]:
results = pd.read_csv("Results/utility/results.csv")
accuracies = results["accuracies"].to_list()
utilities = results["utilities"].to_list()
benchmark = accuracies[0]
# accuracies, utilities = accuracies[1:], utilities[1:]

In [ ]:
fig, ax = plt.subplots(1, figsize=(24, 6))

_ = ax.set_xlabel("utility")
_ = ax.set_xlim([0, 1])
_ = ax.set_ylabel("accuracy")
_ = ax.set_title("Expected utility tradeoff")
_ = ax.axhline(y=benchmark, linewidth=3, linestyle="--", color="gray", label="Benchmark")
_ = ax.plot(utilities, accuracies, 'o',  label="SERM")
_ = ax.legend()